In [ ]:
from funman import Funman, ModelParameter, StructureParameter, ParameterSpace, ParameterSynthesisScenario, ParameterSynthesisScenarioResult, ResultCombinedHandler, FUNMANConfig, BilayerModel, BilayerDynamics, QueryLE

from funman_demo import NotebookImageRefresher, RealtimeResultPlotter, ResultCacheWriter, plot_parameter_space, plot_cached_search

import json
import os

#%matplotlib inline

In [ ]:
# The Funman module synthesizes the Parameter Space for a model, which encodes
# feasible parameter settings (denoted "true"), and infeasible parameter
# settings (denoted "false").  The code in this cell generates the Parameter
# Space and plots it.
#
# Funman synthesizes parameters with a box search algorithm that encodes a
# series of related Satisfiability Modulo Theories (SMT) encodings.  The box
# search involves sub-dividing the space of possible parameter-value
# combinations to identify boxes that either contain only feasible values or
# infeasible values for the parameters.  If a box contains a feasible point, the
# solver attempts to prove the absence of infeasible points.  If no infeasible
# points are found, then the box is labeled feasible.  Similarly, if a box
# contains an infeasible point and no feasible points, it is labeled infeasible.
# If a box contains both a feasible and infeasible point, then the solver
# subdivides the box.  The labeling and subdividing process continues until no
# boxes above a given threshold size remain.
#
# Funman evaluates the extracted model to identify values for beta that:
#  - Represent the intervention efficacy over two successive interventions
#  - Occur over two time periods 0-20 days (beta_1), and 20-30 days (beta_2)
#  - Prevent the number of infected I from exceeding 10% of the population
#  - Use a linearized definition for the S and R state update equations
#  - Fall within a specified range [0.0, 0.5]
#
# The Parameter Synthesis problem accepts two constants: 
#  - population_size: the number of individuals (S+I+R)
#  - infectious_days: the number of infected days before recovery
#
#  The scenario configuration also supports use of:
#  - epochs: the day index ranges corresponding to each beta
#  - read_cache_parameter_space: use a precomputed Parameter Space encoded as JSON
#  - write_cache_parameter_space: write a precomputed Parameter Space encoded as JSON
#  - real_time_plotting: whether to render the solver's progress
parameters = [ModelParameter(name="beta", lb=0.0, ub=0.5), ModelParameter(name="gamma", lb=0.0, ub=0.5),
                StructureParameter(name="num_steps", lb=3, ub=3),
                StructureParameter(name="step_size", lb=1, ub=1)]

bilayer_path = os.path.join(
    "../resources", "bilayer", "CHIME_SIR_dynamics_BiLayer.json"
)
with open(bilayer_path, "r") as f:
    bilayer_src = json.load(f)

infected_threshold = 3
init_values = {"S": 9998, "I": 1, "R": 1}

scale_factor = 0.5
lb = 0.000067 * (1 - scale_factor)
ub = 0.000067 * (1 + scale_factor)

model = BilayerModel(
    bilayer=BilayerDynamics(json_graph=bilayer_src),
    init_values=init_values,
    parameter_bounds={
        "beta": [lb, ub],
        "gamma": [1.0 / 14.0, 1.0 / 14.0],
    },
)

query = QueryLE(variable="I", ub=infected_threshold)


result1 : ParameterSynthesisScenarioResult = Funman().solve(
    ParameterSynthesisScenario(
        parameters=parameters, 
        model=model,
        query=query),
        config = FUNMANConfig(
            tolerance=0.1,
            _wait_action = NotebookImageRefresher("result1.png"),
            _handler = ResultCombinedHandler([
                ResultCacheWriter("result1_cache.json"),
                RealtimeResultPlotter(
                    parameters,
                    plot_points=True,
                    realtime_save_path="result1.png"
                )
            ])
        ))

plot_parameter_space(result1.parameter_space, clear = True)

In [ ]:
plot_cached_search("result1.json")

In [ ]:
# This second use of Funman evaluates the model after changing the population
# size (10x) and infectious days (0.5x) constants.

parameters = [Parameter(name="beta_0", lb=0.0, ub=0.5), Parameter(name="beta_1", lb=0.0, ub=0.5)]
gromet_file2 = "baz"
result2 : ParameterSynthesisScenarioResult = Funman().solve(
    ParameterSynthesisScenario(
        parameters, 
        gromet_file2,
        config = {
            "linearize": True,
            "epochs": [(0, 20), (20, 30)],
            "population_size": 10020,
            "infectious_days": 7.0,
        }),
        SearchConfig(
            wait_action = NotebookImageRefresher("result2.png"),
            handler = ResultCombinedHandler([
                ResultCacheWriter("result2_cache.json"),
                RealtimeResultPlotter(
                    parameters,
                    plot_points=True,
                    realtime_save_path="result2.png"
                )
            ])
        ))

plot_parameter_space(result2.parameter_space, clear = True)

In [ ]:
plot_cached_search("result2.json")

In [ ]:
# Funman produces two parameter spaces for beta_0 and beta_1.  The parameter
# spaces overlap (i.e., have a non-empty intersection) and indicate values for
# the betas that are robust to changes in the model constants.  Similarly, the
# distinct regions (i.e., symmetric difference) denote parameter values where
# the models behave differently.
#
# The plot below illustrates:
# - The parameter space for the original model constants.
# - The parameter space for the modified model constants.
#
# The interpretation of the intersection (dark green, dark red) is that the feasible values for beta
# denote interventions that scale with population size and the number of
# infectious days.  These values warrant further investigation by the analyst.
# The interpretation of the symmetric difference is that the feasible values for
# beta denote interventions that will not scale with population size and the
# number of infectious days.  These values may be discarded by the analyst.

plot_parameter_space(result1.parameter_space)
plot_parameter_space(result2.parameter_space)